In [1]:
# import packages
import boto3
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
# establish connection with S3 
try:
    s3_client = boto3.client('s3')
    s3_resource = boto3.resource('s3')
except 's3Location not found':
    print("s3Location not found")
except 'Invalid credentials':
    print("Invalid credentials")
except 'Invalid region':
    print("Invalid region")
except'No Files were found':
    print("No Files were found")
except 'Invalid File Format' :
    print("Invalid File Format")
except 'No Records Found':
    print("No Records Found")
else:
    print("Connection Established")

Connection Established


In [3]:
# finding keys of click logs from s3 bucket

clickkeys = []
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.endswith(suffix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
for i in get_matching_s3_keys('soka-sample','click'):
    clickkeys.append(i)
print(len(clickkeys))

9743


In [4]:
# finding keys of impression logs from s3 bucket

impressionkeys = []
def get_matching_s3_keys(bucket, prefix='', suffix=''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.endswith(suffix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
for i in get_matching_s3_keys('soka-sample','impression'):
    impressionkeys.append(i)
print(len(impressionkeys))

9999


In [5]:
# forming the dataset comprising of 'SLOT','CH', 'AT','VALUE' from click logs and impression logs

clickpredictiondata = pd.DataFrame(columns=['SLOT','CH','AT','VALUE'])
temp1 = pd.DataFrame(columns=['SLOT','CH','AT','VALUE'])
for i in range(400):
    obj = s3_client.get_object(Bucket='soka-sample', Key=clickkeys[i])
    contents = obj['Body'].read()
    df=pd.read_json(contents, lines=True)
    df = pd.DataFrame(df[['SLOT','CH','AT']])
    temp1 = temp1.append(df)
for i,r in temp1.iterrows():
    temp1.loc[i,'VALUE'] = 1
print(len(temp1))
temp2 = pd.DataFrame(columns=['SLOT','CH','AT','VALUE'])
for i in range(40):
    obj = s3_client.get_object(Bucket='soka-sample', Key=impressionkeys[i])
    contents = obj['Body'].read()
    df=pd.read_json(contents, lines=True)
    df = pd.DataFrame(df[['SLOT','CH','AT']])
    temp2 = temp2.append(df)
for i,r in temp2.iterrows():
    temp2.loc[i,'VALUE'] = 0
print(len(temp2))
clickpredictiondata = clickpredictiondata.append(temp1)
clickpredictiondata = clickpredictiondata.append(temp2)
print(len(clickpredictiondata))

9070
12302
21372


In [6]:
#  Finding dependent and independent variables

dataset = clickpredictiondata
X = pd.DataFrame(dataset.iloc[:, [0,1,2]].values)
y = dataset.iloc[:, 3].values

In [7]:
# encooding categorical columns ('CH', 'AT') step 1

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder() 
X = X.apply(LabelEncoder().fit_transform)
X = X.as_matrix()
print(X)

[[2 0 1]
 [1 0 0]
 [1 0 1]
 ...
 [2 0 7]
 [2 0 8]
 [2 0 9]]


In [8]:
# encooding categorical columns ('CH') step 2

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
onehotencoder = OneHotEncoder(categorical_features = [0,1])
X = onehotencoder.fit_transform(X).toarray()

C:\Users\ashwi\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [9]:
# splitting into training and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [10]:
# # converting datatype of X_train

X_train = np.asarray(X_train, dtype="float")

In [11]:
# converting datatype of y_train

y_train = np.asarray(y_train, dtype="float")

In [12]:
# importing Suppost vector machine model

from sklearn.svm import SVC
classifier = SVC()

In [13]:
# fitting the model

classifier.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [16]:
# predicting the results of X_test using the model

y_pred = classifier.predict(X_test)

In [114]:
# count of correct prediction

cp=0
temp=[]
for i in range(len(y_pred)):
    if(y_pred[i]==y_test[i]):
        cp+=1
print("Correct Prediction",cp)
print("Mis prediction",len(y_pred)-cp)

Correct Prediction 2477
Mis prediction 1167


In [17]:
# Confusion matrix of click probablity with features (slot, channel)

df_confusion = pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
df_confusion

Predicted,0.0,1.0,All
Actual,,,
0,1355,498,1853
1,487,866,1353
All,1842,1364,3206


In [138]:
# predicting a random data

temp = [[1,0,0,1],]
classifier.predict(temp)

array([0.])